## DATA CLEANING AND WRANGLING FOR TABLEAU DASHBOARD

The objective of this project is to compile data from several publicly available data sources from Latvian state instititutions.<br> Despite the great data Tableau processing capabilities it could not join 5 different .xls and.csv tables with geospatial data in .shp file. So the objective of this Python code is to merge the relevant data from these files into a new dataframe and export it for further use in Tableau.<br>Data in the input files  cover such economic and demographic indicators as:
* dynamics of new company registration, 
* business classification(NACE),  
* salary data of paid and self-employed workers,
* population dynamics.

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Latvia Company Register data, 440 000 records on registration of companies in 1990 - 2019.
file1 = 'http://dati.ur.gov.lv/register/register.csv' 
file2 = 'https://www.dropbox.com/s/knfc29x4tmo065k/Novadu_atvk_kodi.xlsx?dl=1' #regional codes needed for geospatial data
file3 = 'https://www.dropbox.com/s/9jqoo6c5yjqy64a/Population_2013-2017.csv?dl=1'#population dynamics 2013 - 2017
file4 = 'https://www.dropbox.com/s/ql9dzkmsmi8sxte/iemaksu_alga_ed.xlsx?dl=1'# salary data 2018
file5 = 'https://www.dropbox.com/s/7ii9dp7kcbt7al6/business_classificator.xlsx?dl=1'#NACE business classificator data

In [3]:
df1 = pd.read_csv(file1, sep=';', low_memory = False, error_bad_lines=False)

In [4]:
df1.shape

(411942, 21)

In [5]:
df1.columns

Index(['regcode', 'sepa', 'name', 'name_before_quotes', 'name_in_quotes',
       'name_after_quotes', 'without_quotes', 'regtype', 'regtype_text',
       'type', 'type_text', 'registered', 'terminated', 'closed', 'address',
       'index', 'addressid', 'region', 'city', 'atvk', 'reregistration_term'],
      dtype='object')

In [6]:
#filtering companies by business type...
dft2 = df1.loc[df1['type'].isin(["SIA", "ZEM"])]

#... and year of registration.
pd.options.mode.chained_assignment = None
dft2['registered'] = pd.to_datetime(dft2['registered'])

dft2 = dft2.loc[dft2['registered']>'2009-01-01 00:00:00']
dft2 = pd.DataFrame(dft2.drop(['sepa', 'name_before_quotes', 'name_in_quotes',
       'name_after_quotes', 'without_quotes', 'regtype', 'regtype_text',
       'type_text', 'closed', 'index', 'addressid', 'region', 'city', 'reregistration_term'], axis=1))

#renaming columns that will be used as table keys later on.
dft2['address'] = dft2['address'].str.split(',', 1).str[0].str.strip()
dft2['address'] = dft2['address'].str.replace(r'nov.', 'novads').str.strip()

dft2=dft2.rename(index=str, columns={'address':'novads', 'atvk':'atvk_ur'})
#assigning year date to a new column.
dft2['year'] = pd.DatetimeIndex(dft2['registered']).year

In [7]:
#creating new dataframe to store registration data by region and year
col = dft2['year'].unique().tolist()
row = dft2['novads'].unique().tolist()

D = np.zeros(shape=(len(row),len(col)))
df_uz = pd.DataFrame(D, columns=col, index=row)
df_uz = df_uz.reindex(sorted(df_uz.columns), axis=1)
df_uz.index.name = 'novads'

#extracting registration data from original table and calculating the new values for each region/year.
x = range(len(dft2.index))

for i in x:
    r1 = dft2.iloc[i, 5]
    c1 = dft2.iloc[i, 7]
    df_uz.loc[r1, c1] += 1 

In [8]:
#calculating summary column for 2009-2018.
df_uz.columns = df_uz.columns.astype(str)    
df_uz['2009_20018_compsum'] =  df_uz[['2009', '2010', '2011', '2012', '2013',
                                     '2014', '2015', '2016', '2017', '2018']].sum(axis=1)
df_uz = df_uz.reset_index()

In [9]:
df_uz.shape

(119, 13)

In [10]:
#opening tand joining the table with regional geo-codes.
df_atvk = pd.read_excel(file2, sheet_name = 'novadi')

In [11]:
df_atvk=df_atvk.rename(index=str, columns={'ATVK ':'atvk','Administratīvās teritorijas':'novads'})
df_atvk['atvk'] = df_atvk['atvk'].astype(str)
df_atvk.iloc[0,0]= '010000'
df_atvk.iloc[1,0] = '050000'
df_atvk.iloc[2,0] = '090000'
df_atvk['atvk'] = '0' + df_atvk['atvk'].astype(str)

In [12]:
df_atvk.head()

,atvk,novads
0,0010000,Rīga
1,0050000,Daugavpils
2,0090000,Jelgava
3,0110000,Jēkabpils
4,0130000,Jūrmala


In [13]:
df_uz = pd.merge(df_uz, df_atvk, on='novads', how='left')

In [14]:
df_uz.head()

,novads,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2009_20018_compsum,atvk
0,Rīga,4432.0,7205.0,10022.0,9459.0,9250.0,8604.0,7508.0,5963.0,5067.0,5265.0,2037.0,72775.0,0010000
1,Valmiera,72.0,113.0,170.0,133.0,130.0,128.0,116.0,84.0,76.0,89.0,28.0,1111.0,0250000
2,Olaines novads,41.0,107.0,146.0,127.0,144.0,123.0,172.0,131.0,89.0,104.0,28.0,1184.0,0801000
3,Salaspils novads,79.0,134.0,182.0,196.0,166.0,168.0,141.0,110.0,91.0,132.0,51.0,1399.0,0801200
4,Jūrmala,179.0,322.0,423.0,421.0,431.0,358.0,332.0,271.0,259.0,261.0,118.0,3257.0,0130000


In [15]:
#opening tand joining the table with population dynamics.
df_iedz = pd.read_csv(file3, skiprows=1, sep=',', error_bad_lines=False)

In [16]:
df_iedz=df_iedz.rename(index=str, columns={'2013 Iedzīvotāju skaits gada sākumā':'2013_pop',
                                           '2014 Iedzīvotāju skaits gada sākumā':'2014_pop',
                                           '2015 Iedzīvotāju skaits gada sākumā':'2015_pop',
                                           '2016 Iedzīvotāju skaits gada sākumā':'2016_pop',
                                           '2017 Iedzīvotāju skaits gada sākumā':'2017_pop',
                                           'Teritoriālā vienība': 'novads'})

#calculating 2013-2017 average
df_iedz['pop_avg'] = df_iedz.loc[:,['2013_pop', '2014_pop', '2015_pop', '2016_pop', '2017_pop']].mean(axis=1)
df_uz = pd.merge(df_uz, df_iedz, on='novads', how='left')
# and indicator: 1000 new companies per 1000 inhabitants that will be used as variable in the dashboard's map.
df_uz['comp_1000pop'] = df_uz['2009_20018_compsum']/df_uz['pop_avg']*1000    

In [17]:
#opening tand joining the table with salary data.
df_wg=pd.read_excel(file4, sheet_name='Dati')

In [18]:
df_wg = pd.DataFrame(df_wg.drop(['PP gads','ATVK kods'],axis=1))
df_wg=df_wg.rename(index=str, columns={'Pilsēta, novads':'novads','Oblig. kopā, skaits':'all_count',
                                       'Oblig. kopā, alga':'all_eur',
                                       'Darba ņēm. kopā, skaits':'workers_count',
                                       'Darba ņēm. kopā, alga':'workers_eur',
                                       'Pašnodarb. kopā, skaits':'selfemp_count',
                                       'Pašnodarb. kopā, alga':'selfemp_eur'})

In [19]:
df_uz = pd.merge(df_uz, df_wg, on='novads', how='left')

In [20]:
#The final table to join contains business classification (NACE) data split by regions.
df_noz=pd.read_excel(file5, sheet_name='nozares')

In [21]:
#filling NaNs in regions ('novadi) column with missing names of the respective regions.
x = range(len(df_noz['Novads_nz']))
a = df_noz.iloc[0, 0]
for i in x:       
    if pd.isnull(df_noz.iloc[i, 0]):
        df_noz.iloc[i, 0] = a
    else:
        a = df_noz.iloc[i, 0]

df_noz = pd.DataFrame(df_noz.drop(['Nod. sk.'],axis=1))

In [22]:
df_noz.columns

Index(['Novads_nz', 'A Lauksaimniecība, mežsaimniecība un zivsaimniecība',
       'B Ieguves rūpniecība un karjeru izstrāde', 'C Apstrādes rūpniecība',
       'D Elektroenerģija, gāzes apgāde, siltumapgāde un gaisa kondicionēš',
       'E Ūdens apgāde; notekūdeņu, atkritumu apsaimniekošana un sanācija',
       'F Būvniecība',
       'G Vairumtirdzniecība un mazumtirdzniecība; automobiļu un motociklu',
       'H Transports un uzglabāšana',
       'I Izmitināšana un ēdināšanas pakalpojumi',
       'J Informācijas un komunikācijas pakalpojumi',
       'K Finanšu un apdrošināšanas darbības',
       'L Operācijas ar nekustamo īpašumu',
       'M Profesionālie, zinātniskie un tehniskie pakalpojumi',
       'N Administratīvo un apkalpojošo dienestu darbība',
       'O Valsts pārvalde un aizsardzība; obligātā sociālā apdrošināšana',
       'P Izglītība', 'Q Veselība un sociālā aprūpe',
       'R Māksla, izklaide un atpūta', 'S Citi pakalpojumi',
       'T Mājsaimniecību kā darba devēju darbība

In [23]:
L = ['A Lauksaimniecība, mežsaimniecība un zivsaimniecība',
       'B Ieguves rūpniecība un karjeru izstrāde', 'C Apstrādes rūpniecība',
       'D Elektroenerģija, gāzes apgāde, siltumapgāde un gaisa kondicionēš',
       'E Ūdens apgāde; notekūdeņu, atkritumu apsaimniekošana un sanācija',
       'F Būvniecība',
       'G Vairumtirdzniecība un mazumtirdzniecība; automobiļu un motociklu',
       'H Transports un uzglabāšana',
       'I Izmitināšana un ēdināšanas pakalpojumi',
       'J Informācijas un komunikācijas pakalpojumi',
       'K Finanšu un apdrošināšanas darbības',
       'L Operācijas ar nekustamo īpašumu',
       'M Profesionālie, zinātniskie un tehniskie pakalpojumi',
       'N Administratīvo un apkalpojošo dienestu darbība',
       'O Valsts pārvalde un aizsardzība; obligātā sociālā apdrošināšana',
       'P Izglītība', 'Q Veselība un sociālā aprūpe',
       'R Māksla, izklaide un atpūta', 'S Citi pakalpojumi',
       'T Mājsaimniecību kā darba devēju darbība; pašpatēriņa preču ražoša',
       'U Ārpusteritoriālo organizāciju un institūciju darbība',
       'NSP Nav noteikts']

In [25]:
#grouping by regions
df_noz[L] = df_noz[L].apply(pd.to_numeric, errors='coerce')
df_noz_gr = df_noz.groupby('Novads_nz').sum()
df_noz_gr = df_noz_gr.reset_index()


In [26]:
df_noz_gr.isnull().sum()

Novads_nz                                                               0
A Lauksaimniecība, mežsaimniecība un zivsaimniecība                     0
B Ieguves rūpniecība un karjeru izstrāde                               36
C Apstrādes rūpniecība                                                  0
D Elektroenerģija, gāzes apgāde, siltumapgāde un gaisa kondicionēš     18
E Ūdens apgāde; notekūdeņu, atkritumu apsaimniekošana un sanācija      42
F Būvniecība                                                            0
G Vairumtirdzniecība un mazumtirdzniecība; automobiļu un motociklu      0
H Transports un uzglabāšana                                             1
I Izmitināšana un ēdināšanas pakalpojumi                                4
J Informācijas un komunikācijas pakalpojumi                             7
K Finanšu un apdrošināšanas darbības                                   19
L Operācijas ar nekustamo īpašumu                                       2
M Profesionālie, zinātniskie un tehnis

In [27]:
# removing columns with high proportion of NaNs except state institutions since they understandably(?)
# concentrate only in few regions.
df_noz_gr = pd.DataFrame(df_noz_gr.drop(['T Mājsaimniecību kā darba devēju darbība; pašpatēriņa preču ražoša',
                                         'U Ārpusteritoriālo organizāciju un institūciju darbība',
                                         'NSP Nav noteikts'],axis=1))

In [28]:
#Renaming columns to English.
df_noz_gr=df_noz_gr.rename(index=str,
                           columns={'A Lauksaimniecība, mežsaimniecība un zivsaimniecība':'Agriculture, Forestry and Fishing',
                                    'B Ieguves rūpniecība un karjeru izstrāde':'Mining and Quarrying',
                                    'C Apstrādes rūpniecība':'Manufacturing',
                                    'D Elektroenerģija, gāzes apgāde, siltumapgāde un gaisa kondicionēš':'Electricity, Gas, Steam and Air Conditioning Supply',
                                    'E Ūdens apgāde; notekūdeņu, atkritumu apsaimniekošana un sanācija':'Water Supply; Sewerage, Waste Management',
                                    'F Būvniecība':'Construction',
                                    'G Vairumtirdzniecība un mazumtirdzniecība; automobiļu un motociklu':'Wholesale and Retail Trade',
                                    'H Transports un uzglabāšana':'Transportation and Storage',
                                    'I Izmitināšana un ēdināšanas pakalpojumi':'Accommodation and Food Service Activities',
                                    'J Informācijas un komunikācijas pakalpojumi':'Information and Communication',
                                    'K Finanšu un apdrošināšanas darbības':'Financial and Insurance Activities',
                                    'L Operācijas ar nekustamo īpašumu':'Real Estate Activities',
                                    'M Profesionālie, zinātniskie un tehniskie pakalpojumi':'Professional, Scientific and Technical Activities',
                                    'N Administratīvo un apkalpojošo dienestu darbība':'Administrative and Support Service Activities',
                                    'O Valsts pārvalde un aizsardzība; obligātā sociālā apdrošināšana':'Public Administration and Defence',
                                    'P Izglītība':'Education',
                                    'Q Veselība un sociālā aprūpe':'Human Health and Social Work Activities',
                                    'R Māksla, izklaide un atpūta':'Arts, Entertainment and Recreation',
                                    'S Citi pakalpojumi':'Other Service Activities',
                                    'Novads_nz':'novads'
                                    })

In [29]:
# creating the final table and exporting it to excel.
df_uz = pd.merge(df_uz, df_noz_gr, on='novads', how='left')

In [30]:
df_uz.head()

,novads,2009,2010,2011,2012,2013,2014,2015,2016,2017,...,Information and Communication,Financial and Insurance Activities,Real Estate Activities,"Professional, Scientific and Technical Activities",Administrative and Support Service Activities,Public Administration and Defence,Education,Human Health and Social Work Activities,"Arts, Entertainment and Recreation",Other Service Activities
0,Rīga,4432.0,7205.0,10022.0,9459.0,9250.0,8604.0,7508.0,5963.0,5067.0,...,4571.0,1467.0,8081.0,11485.0,4104.0,4.0,1696.0,2216.0,2389.0,7836.0
1,Valmiera,72.0,113.0,170.0,133.0,130.0,128.0,116.0,84.0,76.0,...,85.0,22.0,149.0,238.0,59.0,1.0,57.0,108.0,68.0,442.0
2,Olaines novads,41.0,107.0,146.0,127.0,144.0,123.0,172.0,131.0,89.0,...,41.0,21.0,58.0,131.0,66.0,NaN,38.0,25.0,38.0,129.0
3,Salaspils novads,79.0,134.0,182.0,196.0,166.0,168.0,141.0,110.0,91.0,...,67.0,7.0,72.0,183.0,86.0,NaN,45.0,111.0,66.0,230.0
4,Jūrmala,179.0,322.0,423.0,421.0,431.0,358.0,332.0,271.0,259.0,...,170.0,50.0,700.0,522.0,223.0,NaN,112.0,156.0,137.0,436.0


In [31]:
df_uz.to_excel('companydata_table.xlsx', sheet_name='reg_data')

## RESULTS

[Here's the dashboard built on the compiled data!](https://public.tableau.com/profile/sergejs1574#!/vizhome/NewbusinessopeningsinLatviav_2/BusinessopeningsLV)